In [3]:
import numpy as np
from stl import mesh

# Define the 8 vertices of the cube*
vertices = np.array([\
    [-1, -1, -1],
    [+1, -1, -1],
    [+1, +1, -1],
    [-1, +1, -1],
    [-1, -1, +1],
    [+1, -1, +1],
    [+1, +1, +1],
    [-1, +1, +1]])
# Define the 12 triangles composing the cube
faces = np.array([\
    [0,3,1],
    [1,3,2],
    [0,4,7],
    [0,7,3],
    [4,5,6],
    [4,6,7],
    [5,1,2],
    [5,2,6],
    [2,3,6],
    [3,7,6],
    [0,1,5],
    [0,5,4]])

# Create the mesh*
cube = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
for i, f in enumerate(faces):
    for j in range(3):
        print(vertices[f[j],:])
        cube.vectors[i][j] = vertices[f[j]]

# Write the mesh to file "cube.stl"
cube.save('cube.stl')

[-1 -1 -1]
[-1  1 -1]
[ 1 -1 -1]
[ 1 -1 -1]
[-1  1 -1]
[ 1  1 -1]
[-1 -1 -1]
[-1 -1  1]
[-1  1  1]
[-1 -1 -1]
[-1  1  1]
[-1  1 -1]
[-1 -1  1]
[ 1 -1  1]
[1 1 1]
[-1 -1  1]
[1 1 1]
[-1  1  1]
[ 1 -1  1]
[ 1 -1 -1]
[ 1  1 -1]
[ 1 -1  1]
[ 1  1 -1]
[1 1 1]
[ 1  1 -1]
[-1  1 -1]
[1 1 1]
[-1  1 -1]
[-1  1  1]
[1 1 1]
[-1 -1 -1]
[ 1 -1 -1]
[ 1 -1  1]
[-1 -1 -1]
[ 1 -1  1]
[-1 -1  1]


In [7]:
import bpy
import numpy as np

from PIL import Image
grey_img = Image.open("house1.jpg").convert('L')

max_size=(500,500)
max_height=30
min_height=0

grey_img.thumbnail(max_size)
imageNp = np.array(grey_img)
maxPix=imageNp.max()
minPix=imageNp.min()

(W,H)=grey_img.size

print(f"W,H={W},{H}")
vertices=[]
edges=[]
faces=[]

DX=1
DY=1

for Y in range(0, H, DY):
    for X in range(0,W,DX):
        pixelIntensity = imageNp[Y][X]
        Z = (pixelIntensity * max_height) / maxPix
        vertices.append((X,Y,Z))
        
        
for X in range(0, W-1, DX):
    for Y in range(0, H-1, DY):
        face_v1= X+Y*W
        face_v2=X + 1 + Y* W
        face_v3=X + 1 + (Y+1) * W
        
        faces.append((face_v1,face_v2,face_v3))
        
        face_v1= X+Y*W
        face_v2=X  + (Y+1)* W
        face_v3=X + 1 + (Y+1) * W
        
        faces.append((face_v1,face_v2,face_v3))
        
new_mesh=bpy.data.meshes.new("new_mesh")
new_mesh.from_pydata(vertices,edges,faces)
new_mesh.update()
# make objec from the mesh
new_object = bpy.data.objects.new("new_object",new_mesh)
view_layer=bpy.context.view_layer
view_layer.active_layer_collection.collection.objects.link(new_object)



W,H=384,288


In [8]:
grey_img = Image.open("house1.jpg").convert('L')

max_size=(500,500)
max_height=10
min_height=0

#height=0 for minPix
#height=maxHeight for maxPIx

grey_img.thumbnail(max_size)
imageNp = np.array(grey_img)
maxPix=imageNp.max()
minPix=imageNp.min()



print(imageNp)
(ncols,nrows)=grey_img.size

vertices=np.zeros((nrows,ncols,3))

for x in range(0, ncols):
  for y in range(0, nrows):
    pixelIntensity = imageNp[y][x]
    z = (pixelIntensity * max_height) / maxPix
    #print(imageNp[y][x])
    vertices[y][x]=(x, y, z)

faces=[]

for x in range(0, ncols - 1):
  for y in range(0, nrows - 1):
    # create face 1
    vertice1 = vertices[y][x]
    vertice2 = vertices[y+1][x]
    vertice3 = vertices[y+1][x+1]
    face1 = np.array([vertice1,vertice2,vertice3])

    # create face 2 
    vertice1 = vertices[y][x]
    vertice2 = vertices[y][x+1]
    vertice3 = vertices[y+1][x+1]

    face2 = np.array([vertice1,vertice2,vertice3])

    faces.append(face1)
    faces.append(face2)

print(f"number of faces: {len(faces)}")
facesNp = np.array(faces)
# Create the mesh
surface = mesh.Mesh(np.zeros(facesNp.shape[0], dtype=mesh.Mesh.dtype))
for i, f in enumerate(faces):
    for j in range(3):
        surface.vectors[i][j] = facesNp[i][j]
# Write the mesh to file "cube.stl"
surface.save('surface.stl')
print(surface)

[[  6  67 128 ... 124 122 126]
 [  6  68 128 ... 122 127 126]
 [  5  69 129 ... 128 126 128]
 ...
 [  3  59 111 ... 137 139 140]
 [  9  59 107 ...  73  75  77]
 [  3   3   8 ...   4   5   5]]
number of faces: 219842
<Mesh: '' 219842 vertices>


In [51]:
from scipy import spatial
import itertools

vertices = np.array([\
    [-4, -1, -1],
    [+4, -1, -1],
    [+2, +1, -1],
    [-2, +1, -1],
    [-2, -1, +1],
    [+2, -1, +1],
    [+2, +1, +1],
    [-2, +1, +1]])

faces=[]

tree = spatial.KDTree(vertices)

for vertex in vertices:
    for inds in (list(itertools.combinations(tree.query(vertex, k=5)[1],3))):
        faces.append(np.array([vertices[inds[0]],vertices[inds[1]],vertices[inds[2]]]))


print(f"number of faces: {len(faces)}")
facesNp = np.array(faces)
# Create the mesh
surface = mesh.Mesh(np.zeros(facesNp.shape[0], dtype=mesh.Mesh.dtype))
for i, f in enumerate(faces):
    for j in range(3):
        surface.vectors[i][j] = facesNp[i][j]
# Write the mesh to file "cube.stl"
surface.save('surface.stl')
print(surface)

number of faces: 80
<Mesh: '' 80 vertices>


In [ ]:
for i in range(len(vertices)-1):
        plane_a=[]
        plane_b=[]
        for j in range(len(point_list[i])-1):
            tri_a=[]
            tri_b=[]
            #series a triangles
            tri_a.append(point_list[i+1][j])
            tri_a.append(point_list[i][j+1])
            tri_a.append(point_list[i][j])
            #series b triangles
            tri_b.append(point_list[i+1][j])
            tri_b.append(point_list[i+1][j+1])
            tri_b.append(point_list[i][j+1])
            #load to plane
            plane_a.append(tri_a)
            plane_b.append(tri_b)
        group_a.append(plane_a)
        group_b.append(plane_b)


num_triangles=len(fin_list)
data = np.zeros(num_triangles, dtype=mesh.Mesh.dtype)
for i in range(num_triangles):
    #I did not know how to use numpy-arrays in this case. This was the major roadblock
    # assign vertex co-ordinates to variables to write into mesh
    data["vectors"][i] = np.array([[v1x, v1y, v1z],[v2x, v2y, v2z],[v3x, v3y, v3z]])
m=mesh.Mesh(data)
m.save('filename.stl')